In [59]:
!pip install mrjob


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
!pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [61]:
import sys
sys.argv

['/usr/local/lib/python3.9/dist-packages/ipykernel_launcher.py',
 '-f',
 '/root/.local/share/jupyter/runtime/kernel-7726487a-f30f-43be-b9d4-4d713203ee77.json']

In [62]:

# Mapper function
def mapper():
    for line in sys.stdin:
        # Split the line into words
        words = line.strip().split()
        # Emit each word with count of 1
        for word in words:
            print(word + "\t1")

# Reducer function
def reducer():
    current_word = None
    word_count = 0
    for line in sys.stdin:
        # Split the line into word and count
        word, count = line.strip().split("\t")
        count = int(count)
        # If it's the same word, increment the count
        if current_word == word:
            word_count += count
        else:
            # If it's a new word, emit the previous word count
            if current_word:
                print(current_word + "\t" + str(word_count))
            current_word = word
            word_count = count
    # Emit the last word count
    if current_word:
        print(current_word + "\t" + str(word_count))

if __name__ == '__main__':
    # Determine if it's a mapper or reducer task based on command line arguments
    if sys.argv[1] == 'mapper':
        mapper()
    elif sys.argv[1] == 'reducer':
        reducer()


In [63]:
count=1

def mapper(each_rating):
    global count
    counter = {}
    for key in each_rating:
        if key=="text":
            d=[]
            for word in each_rating[key].split(" "):
                p=[]
                url = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))')
                retw=re.compile("(?<!RT\s)@\S+")
                if url.match(word) or word.startswith('#') or retw.match(word) or word=="RT" or word.startswith('@') or word.startswith("rating"):
                    continue
                word=word.lower()
                word=word.encode('UTF-8').translate(None,string.punctuation)
                if word=="":
                    continue
                if word in counter:
                    counter[word] += 1
                else:
                    counter[word] = 1

    for word in counter:
        mr.emit_intermediate(word, (count, counter[word]))

    count=count+1


def reducer(key, list_of_values):
    total = 0
    li = []
    for id, value in list_of_values:
        total += 1
        li.append((id, value))
    mr.emit((key, total, li))
     #inputdata = open("/content/Women Dresses Reviews Dataset  [MConverter.eu].txt")
    inputdata = open(sys.argv[1])
    mr.execute(inputdata, mapper, reducer)





In [64]:
import json

class MapReduce:
    def __init__(self):
        # initialize dictionary for intermediate values from Map task
        self.intermediate = {}
        # initialize list for results of Reduce task
        self.result = []

    def emit_intermediate(self, key, value):
        # if key not already in dictionary, set value to empty list
        self.intermediate.setdefault(key, [])
        # add value to list associated with key
        self.intermediate[key].append(value)

    def emit(self, value):
        # append value to list of results
        self.result.append(value) 

    def execute(self, data, mapper, reducer):
        # read each line from input file; call Map function on each record
        for line in data:
            record = json.loads(line)
            mapper(record)
        
        # for each key:valuelist in intermediate dictionary, call Reduce task
        for key in self.intermediate:
            reducer(key, self.intermediate[key])

        jenc = json.JSONEncoder()
        self.result.sort()
        # print each result in result list
        for item in self.result:
            print (jenc.encode(item))

In [72]:
import pandas as pd
data = pd.read_csv("Amazon.csv")


In [79]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import string
import re
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [74]:
data = data[["title", "rating"]]
print(data.head())
print(data.tail())

                    title  rating
0    Amazing fit and wash       5
1      Lovely and unique!       5
2                     Meh       1
3                     Wow       5
4  Great for bigger busts       5
                                       title  rating
23481                               Love it!       5
23482                            Great piece       5
23483                 So soft and flattering       5
23484  Another winner from isabella sinclair       5
23485                                    NaN       5


In [75]:
print(data.isnull().sum())

title     3810
rating       0
dtype: int64


In [77]:
data = data.dropna()

In [78]:
ratings = data["rating"].value_counts()
numbers = ratings.index
quantity = ratings.values

import plotly.express as px

figure = px.pie(data, 
                values=quantity, 
                names=numbers,hole = 0.5)
figure.show()

In [97]:
import re

from nltk.corpus import stopwords # Import the stop word list

def description_to_words(review_text):

    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text)
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))

In [99]:
nltk.download('vader_lexicon')
sentiments = SentimentIntensityAnalyzer()
data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["title"]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["title"]]
data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["title"]]
data = data[["title", "Positive", "Negative", "Neutral"]]
print(data.head())

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


                    title  Positive  Negative  Neutral
0    Amazing fit and wash     0.759       0.0    0.241
1      Lovely and unique!     0.672       0.0    0.328
2                     Meh     0.000       1.0    0.000
3                     Wow     1.000       0.0    0.000
4  Great for bigger busts     0.577       0.0    0.423


In [92]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Instantiate the sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Define a function to perform sentiment analysis
def analyze_sentiment(text):
    """
    Analyze sentiment of text using SentimentIntensityAnalyzer from NLTK.
    Returns a dictionary with compound, positive, negative, and neutral scores.
    """
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores

# Example text
review_text = "The food at the restaurant was delicious, but the service was slow."

# Perform sentiment analysis on the review text
sentiment_scores = analyze_sentiment(review_text)

# Extract sentiment scores
compound_score = sentiment_scores['compound']
positive_score = sentiment_scores['pos']
negative_score = sentiment_scores['neg']
neutral_score = sentiment_scores['neu']

# Print sentiment scores
print("Compound Score: ", compound_score)
print("Positive Score: ", positive_score)
print("Negative Score: ", negative_score)
print("Neutral Score: ", neutral_score)

# Interpret sentiment
if compound_score > 0:
    print("Overall sentiment: Positive")
elif compound_score < 0:
    print("Overall sentiment: Negative")
else:
    print("Overall sentiment: Neutral")


Compound Score:  0.3291
Positive Score:  0.176
Negative Score:  0.0
Neutral Score:  0.824
Overall sentiment: Positive
